<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    55
1     9
Name: InPortfolio, dtype: int64

In [6]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    stock_df['Trend'] = 'Down'
    stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    stock_df['EOM'] = '-ve'
    stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5',
                   'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    #print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [8]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM
Date,,,,,,,,,,,,
2022-12-22 00:00:00+05:30,18127.0,^NSEI,5.53,0.44,0.58,18812.0,15294.0,80.54,19.46,Down,40.0,-ve
2022-12-23 00:00:00+05:30,17807.0,^NSEI,3.62,0.45,0.54,18812.0,15294.0,71.43,28.57,Down,32.0,-ve
2022-12-26 00:00:00+05:30,18015.0,^NSEI,4.76,0.48,0.53,18812.0,15294.0,77.33,22.67,Down,41.0,-ve
2022-12-27 00:00:00+05:30,18132.0,^NSEI,5.38,0.50,0.52,18812.0,15294.0,80.68,19.32,Down,45.0,-ve
2022-12-28 00:00:00+05:30,18122.0,^NSEI,5.27,0.51,0.50,18812.0,15294.0,80.40,19.60,Down,45.0,-ve


In [9]:
# MACD Uptrend
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [10]:
df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)

,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
53,2022-12-28 00:00:00+05:30,3446.0,DIVISLAB.NS,-8.46,-0.9,-0.2,4643.0,3266.0,13.1,86.9,Up,54.0,+ve,3560.0,3.0


In [11]:
# portfolio condition
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
1,2022-12-28 00:00:00+05:30,449.0,ICICIPRULI.NS,-12.90,-0.19,-1.47,615.0,440.0,5.40,94.60,Down,41.0,-ve,640.0,43.0
2,2022-12-28 00:00:00+05:30,693.0,MOTILALOFS.NS,-10.08,-1.04,-0.15,960.0,654.0,12.83,87.17,Down,53.0,+ve,930.0,34.0
3,2022-12-28 00:00:00+05:30,1242.0,SBILIFE.NS,4.29,0.76,0.32,1332.0,1023.0,70.89,29.11,Down,46.0,-ve,1607.0,29.0
10,2022-12-28 00:00:00+05:30,568.0,HDFCLIFE.NS,2.38,0.54,0.72,672.0,502.0,38.94,61.06,Down,48.0,-ve,681.0,20.0
13,2022-12-28 00:00:00+05:30,3933.0,NAUKRI.NS,-3.33,-0.44,0.45,5667.0,3363.0,24.75,75.25,Down,44.0,-ve,4677.0,19.0
20,2022-12-28 00:00:00+05:30,1510.0,INFY.NS,-1.55,-0.59,0.51,1898.0,1351.0,29.10,70.90,Down,41.0,-ve,1715.0,14.0
22,2022-12-28 00:00:00+05:30,1078.0,MUTHOOTFIN.NS,-3.04,-1.26,0.37,1517.0,956.0,21.77,78.23,Down,48.0,-ve,1217.0,13.0
26,2022-12-28 00:00:00+05:30,1016.0,TECHM.NS,-7.86,-1.63,0.10,1750.0,935.0,9.97,90.03,Down,43.0,-ve,1139.0,12.0
53,2022-12-28 00:00:00+05:30,3446.0,DIVISLAB.NS,-8.46,-0.90,-0.20,4643.0,3266.0,13.10,86.90,Up,54.0,+ve,3560.0,3.0


In [12]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 2


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
6,2022-12-28 00:00:00+05:30,514.0,JUBLFOOD.NS,-7.63,-0.36,-1.43,785.0,463.0,15.99,84.01,Down,38.0,-ve,643.0,25.0
11,2022-12-28 00:00:00+05:30,1111.0,HAVELLS.NS,-9.54,0.00,-1.17,1400.0,1068.0,12.89,87.11,Down,29.0,-ve,1328.0,20.0


In [13]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     22.994652
2     18.181818
3     15.508021
10    10.695187
13    10.160428
20     7.486631
22     6.951872
26     6.417112
53     1.604278
Name: Upside%, dtype: float64

In [14]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
0,2022-12-28 00:00:00+05:30,435.0,GODREJIND.NS,-4.40,-0.66,0.00,644.0,405.0,12.55,87.45,Down,43.0,-ve,638.0,47.0
4,2022-12-28 00:00:00+05:30,239.0,ZEEL.NS,-4.76,0.40,-1.16,325.0,206.0,27.77,72.23,Down,32.0,-ve,303.0,27.0
5,2022-12-28 00:00:00+05:30,391.0,TATAMOTORS.NS,-8.57,-0.47,-0.24,525.0,372.0,12.61,87.39,Down,37.0,-ve,495.0,27.0
6,2022-12-28 00:00:00+05:30,514.0,JUBLFOOD.NS,-7.63,-0.36,-1.43,785.0,463.0,15.99,84.01,Down,38.0,-ve,643.0,25.0
7,2022-12-28 00:00:00+05:30,6476.0,BAJFINANCE.NS,-3.05,-0.07,-1.15,7824.0,5265.0,47.33,52.67,Down,39.0,-ve,8015.0,24.0
8,2022-12-28 00:00:00+05:30,899.0,ICICIBANK.NS,10.71,1.23,0.22,953.0,650.0,82.16,17.84,Down,44.0,-ve,1094.0,22.0
9,2022-12-28 00:00:00+05:30,3212.0,EICHERMOT.NS,5.06,1.31,-0.84,3850.0,2241.0,60.32,39.68,Down,41.0,-ve,3884.0,21.0


In [15]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 85]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
0,2022-12-28 00:00:00+05:30,435.0,GODREJIND.NS,-4.40,-0.66,0.00,644.0,405.0,12.55,87.45,Down,43.0,-ve,638.0,47.0
5,2022-12-28 00:00:00+05:30,391.0,TATAMOTORS.NS,-8.57,-0.47,-0.24,525.0,372.0,12.61,87.39,Down,37.0,-ve,495.0,27.0
11,2022-12-28 00:00:00+05:30,1111.0,HAVELLS.NS,-9.54,0.00,-1.17,1400.0,1068.0,12.89,87.11,Down,29.0,-ve,1328.0,20.0
28,2022-12-28 00:00:00+05:30,208.0,TATAPOWER.NS,-8.66,-0.44,-0.45,287.0,195.0,14.40,85.60,Down,40.0,-ve,232.0,12.0
33,2022-12-28 00:00:00+05:30,591.0,BERGEPAINT.NS,-6.48,-0.47,-0.50,787.0,558.0,14.43,85.57,Down,43.0,-ve,651.0,10.0
35,2022-12-28 00:00:00+05:30,388.0,WIPRO.NS,-12.66,-2.03,-0.25,714.0,375.0,3.78,96.22,Down,44.0,-ve,425.0,10.0
46,2022-12-28 00:00:00+05:30,4332.0,LTIM.NS,-7.99,-1.70,-0.62,7506.0,3796.0,14.45,85.55,Down,39.0,-ve,4654.0,7.0


In [16]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Premium%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Dev%_200', ascending=True)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
32,2022-12-28 00:00:00+05:30,1630.0,HDFCBANK.NS,12.40,0.83,1.27,1662.0,1281.0,91.55,8.45,Down,56.0,+ve,1813.0,11.0
36,2022-12-28 00:00:00+05:30,2671.0,HDFC.NS,12.81,0.84,1.35,2729.0,2053.0,91.46,8.54,Down,54.0,+ve,2940.0,10.0
60,2022-12-28 00:00:00+05:30,2121.0,LT.NS,17.25,1.16,0.97,2195.0,1453.0,90.04,9.96,Down,53.0,-ve,2036.0,-4.0
62,2022-12-28 00:00:00+05:30,4386.0,BRITANNIA.NS,20.29,1.65,1.44,4526.0,2995.0,90.85,9.15,Down,56.0,-ve,3935.0,-10.0
16,2022-12-28 00:00:00+05:30,925.0,AXISBANK.NS,21.03,1.31,1.34,950.0,625.0,92.22,7.78,Down,55.0,+ve,1060.0,15.0
24,2022-12-28 00:00:00+05:30,134.0,FEDERALBNK.NS,22.61,1.83,0.75,138.0,80.0,92.50,7.50,Down,51.0,-ve,152.0,13.0
44,2022-12-28 00:00:00+05:30,110.0,TATASTEEL.NS,27.91,2.33,0.93,116.0,44.0,91.67,8.33,Down,53.0,-ve,118.0,7.0
55,2022-12-28 00:00:00+05:30,316.0,CANBK.NS,32.26,2.09,2.61,329.0,175.0,91.62,8.38,Down,53.0,-ve,322.0,2.0
59,2022-12-28 00:00:00+05:30,182.0,BANKBARODA.NS,44.29,3.17,2.41,193.0,78.0,90.26,9.74,Down,58.0,-ve,183.0,1.0


In [17]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
43,2022-12-28 00:00:00+05:30,1035.0,HCLTECH.NS,2.62,-0.30,0.66,1291.0,865.0,40.01,59.99,Down,41.0,-ve,1117.0,8.0
52,2022-12-28 00:00:00+05:30,3257.0,TCS.NS,-1.33,-0.36,0.49,3965.0,2974.0,28.57,71.43,Down,46.0,-ve,3385.0,4.0
5,2022-12-28 00:00:00+05:30,391.0,TATAMOTORS.NS,-8.57,-0.47,-0.24,525.0,372.0,12.61,87.39,Down,37.0,-ve,495.0,27.0
28,2022-12-28 00:00:00+05:30,208.0,TATAPOWER.NS,-8.66,-0.44,-0.45,287.0,195.0,14.40,85.60,Down,40.0,-ve,232.0,12.0
33,2022-12-28 00:00:00+05:30,591.0,BERGEPAINT.NS,-6.48,-0.47,-0.50,787.0,558.0,14.43,85.57,Down,43.0,-ve,651.0,10.0
47,2022-12-28 00:00:00+05:30,4018.0,DMART.NS,-0.50,-0.17,-0.78,4731.0,3231.0,52.45,47.55,Down,48.0,+ve,4282.0,7.0
7,2022-12-28 00:00:00+05:30,6476.0,BAJFINANCE.NS,-3.05,-0.07,-1.15,7824.0,5265.0,47.33,52.67,Down,39.0,-ve,8015.0,24.0
6,2022-12-28 00:00:00+05:30,514.0,JUBLFOOD.NS,-7.63,-0.36,-1.43,785.0,463.0,15.99,84.01,Down,38.0,-ve,643.0,25.0


In [18]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 4


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
21,2022-12-28 00:00:00+05:30,79.0,NATIONALUM.NS,-6.25,-3.57,1.33,128.0,67.0,19.26,80.74,Down,56.0,-ve,90.0,14.0
38,2022-12-28 00:00:00+05:30,466.0,HINDALCO.NS,7.36,-1.38,1.14,626.0,314.0,48.70,51.30,Down,56.0,-ve,514.0,10.0
43,2022-12-28 00:00:00+05:30,1035.0,HCLTECH.NS,2.62,-0.30,0.66,1291.0,865.0,40.01,59.99,Down,41.0,-ve,1117.0,8.0
52,2022-12-28 00:00:00+05:30,3257.0,TCS.NS,-1.33,-0.36,0.49,3965.0,2974.0,28.57,71.43,Down,46.0,-ve,3385.0,4.0


In [19]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
55,2022-12-28 00:00:00+05:30,316.0,CANBK.NS,32.26,2.09,2.61,329.0,175.0,91.62,8.38,Down,53.0,-ve,322.0,2.0
36,2022-12-28 00:00:00+05:30,2671.0,HDFC.NS,12.81,0.84,1.35,2729.0,2053.0,91.46,8.54,Down,54.0,+ve,2940.0,10.0
16,2022-12-28 00:00:00+05:30,925.0,AXISBANK.NS,21.03,1.31,1.34,950.0,625.0,92.22,7.78,Down,55.0,+ve,1060.0,15.0
32,2022-12-28 00:00:00+05:30,1630.0,HDFCBANK.NS,12.40,0.83,1.27,1662.0,1281.0,91.55,8.45,Down,56.0,+ve,1813.0,11.0
49,2022-12-28 00:00:00+05:30,7054.0,ULTRACEMCO.NS,10.46,0.66,1.23,7825.0,5148.0,71.19,28.81,Down,52.0,-ve,7420.0,5.0
41,2022-12-28 00:00:00+05:30,2015.0,INDIGO.NS,9.22,0.60,1.22,2265.0,1549.0,65.11,34.89,Down,57.0,+ve,2176.0,8.0
37,2022-12-28 00:00:00+05:30,569.0,DABUR.NS,4.43,0.37,0.71,604.0,480.0,71.90,28.10,Down,45.0,-ve,625.0,10.0
29,2022-12-28 00:00:00+05:30,2544.0,RELIANCE.NS,-0.37,0.47,0.62,2811.0,2229.0,54.20,45.80,Down,43.0,-ve,2854.0,12.0
58,2022-12-28 00:00:00+05:30,877.0,MCDOWELL-N.NS,4.18,0.36,0.56,947.0,732.0,67.53,32.47,Down,39.0,-ve,885.0,1.0


In [20]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Trend,RSI,EOM,Forecast,Upside%
21,2022-12-28 00:00:00+05:30,79.0,NATIONALUM.NS,-6.25,-3.57,1.33,128.0,67.0,19.26,80.74,Down,56.0,-ve,90.0,14.0
38,2022-12-28 00:00:00+05:30,466.0,HINDALCO.NS,7.36,-1.38,1.14,626.0,314.0,48.70,51.30,Down,56.0,-ve,514.0,10.0
43,2022-12-28 00:00:00+05:30,1035.0,HCLTECH.NS,2.62,-0.30,0.66,1291.0,865.0,40.01,59.99,Down,41.0,-ve,1117.0,8.0
52,2022-12-28 00:00:00+05:30,3257.0,TCS.NS,-1.33,-0.36,0.49,3965.0,2974.0,28.57,71.43,Down,46.0,-ve,3385.0,4.0
0,2022-12-28 00:00:00+05:30,435.0,GODREJIND.NS,-4.40,-0.66,0.00,644.0,405.0,12.55,87.45,Down,43.0,-ve,638.0,47.0
12,2022-12-28 00:00:00+05:30,80.0,IBREALEST.NS,-0.74,-1.23,0.00,174.0,60.0,17.89,82.11,Down,47.0,-ve,95.0,19.0
50,2022-12-28 00:00:00+05:30,81.0,SAIL.NS,-0.37,-1.23,0.00,108.0,63.0,39.33,60.67,Down,47.0,-ve,85.0,5.0
5,2022-12-28 00:00:00+05:30,391.0,TATAMOTORS.NS,-8.57,-0.47,-0.24,525.0,372.0,12.61,87.39,Down,37.0,-ve,495.0,27.0
35,2022-12-28 00:00:00+05:30,388.0,WIPRO.NS,-12.66,-2.03,-0.25,714.0,375.0,3.78,96.22,Down,44.0,-ve,425.0,10.0
46,2022-12-28 00:00:00+05:30,4332.0,LTIM.NS,-7.99,-1.70,-0.62,7506.0,3796.0,14.45,85.55,Down,39.0,-ve,4654.0,7.0
